In [1]:
! pip install python-igraph
!pip install pyvis
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import re
import csv
import json
import rdflib
import gensim
import pandas as pd
from gensim import corpora
from pymongo import MongoClient
from nltk.corpus import stopwords
import plotly.graph_objects as go
from pyvis.network import Network
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [4]:
client = MongoClient("mongodb://localhost:27017/")
db = client['DataTails']
collection = db['Data']  
data_cursor = collection.find({})
DF = pd.DataFrame(list(data_cursor))
print(DF.head())

                        _id  type  \
0  66e965e698330736e0d693d5  None   
1  66e965e698330736e0d693d6  None   
2  66e965e698330736e0d693d7  None   
3  66e965e698330736e0d693d8  None   
4  66e965e698330736e0d693d9  None   

                                           postTitle postDesc  \
0  Adults(especially those over 30), how young do...      NaN   
1  What is a thing that your parents consider nor...      NaN   
2                 What is a smell that comforts you?      NaN   
3  When in history was it the best time to be a w...      NaN   
4    What's the worst way someone broke up with you?      NaN   

              postTime            authorName noOfUpvotes isNSFW  \
0  2024-08-06 01:02:35  Excellent-Studio7257        4068  False   
1  2024-08-06 01:47:22        Bigbumoffhappy        2073  False   
2  2024-08-05 22:21:53         bloomsmittenn        2188  False   
3  2024-08-06 03:32:59   More_food_please_77         778  False   
4  2024-08-05 21:01:39    ImpressiveWrap7363       

In [ ]:
lemmatizer = WordNetLemmatizer()
StopWords = set(stopwords.words('english'))
def Preprocessing(text):
    text = re.sub(r'\W', ' ', text)
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in StopWords]
    return ' '.join(tokens)

Cols = ['subReddit', 'postTitle', 'postDesc', 'postTime', 'authorName', 'noOfUpvotes', 'comments', 'noOfComments', 'postUrl']
DF = DF[Cols]
print(DF.isnull().sum())
DF['postDesc'].fillna('', inplace=True)
DF['noOfUpvotes'].fillna(0, inplace=True)
DF['postTime'] = pd.to_datetime(DF['postTime'], errors='coerce')
DF['comments'] = DF['comments'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
DF['postTitle'] = DF['postTitle'].apply(lambda x: Preprocessing(str(x)))
DF['postDesc'] = DF['postDesc'].apply(lambda x: Preprocessing(str(x)))
print(DF.head())
print(DF.isnull().sum())

In [ ]:
GroupedData = DF.groupby('subReddit')
SubReddits = defaultdict(dict)
for subreddit, group in GroupedData:
    print(f"Processing Subreddit: {subreddit}")
    group['Combined'] = group['postTitle'] + " " + group['postDesc']
    group['Tokens'] = group['Combined'].apply(lambda x: word_tokenize(x))
    dictionary = corpora.Dictionary(group['Tokens'])
    corpus = [dictionary.doc2bow(text) for text in group['Tokens']]
    LDA = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)
    SubReddits[subreddit]['lda_model'] = LDA
    SubReddits[subreddit]['dictionary'] = dictionary
    SubReddits[subreddit]['corpus'] = corpus
    group['topics'] = [LDA.get_document_topics(bow) for bow in corpus]
    DF.loc[group.index, 'topics'] = group['topics']

In [ ]:
print(DF.shape)
print(DF.describe())
print(DF.head())


In [5]:
lemmatizer = WordNetLemmatizer()
StopWords = set(stopwords.words('english'))
custom_stopwords = StopWords | {"make", "thing", "know", "http", "get", "want", "like", "would", "could", "you", "say","also","aita","com","www","made","ago","day","000"}
def Preprocessing(text):
    text = re.sub(r'\W', ' ', text)  # Remove non-alphanumeric characters
    text = re.sub(r'\b\w{1,2}\b', '', text)  # Remove short words
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in custom_stopwords and len(word) > 2]
    return tokens

Cols = ['subReddit', 'postTitle', 'postDesc', 'postTime', 'authorName', 'noOfUpvotes', 'comments', 'noOfComments', 'postUrl','imageUrl','isNSFW']
DF = DF[Cols]
print(DF.isnull().sum())


DF['subReddit'] = DF['subReddit'].fillna('Unknown_SubReddit')
DF['authorName'] = DF['authorName'].fillna('Unknown_Author')
DF['postTitle'] = DF['postTitle'].fillna('Untitled')
DF['postUrl'] = DF['postUrl'].fillna('http://example.com/NOPOST.png')
DF['imageUrl'] = DF['imageUrl'].fillna('http://example.com/NOImage.png')
DF['isNSFW'] = DF['isNSFW'].fillna(False)
DF['postDesc'].fillna('', inplace=True)
DF['noOfUpvotes'].fillna(0, inplace=True)
DF['noOfComments'].fillna(0, inplace=True)
DF['postTime'] = pd.to_datetime(DF['postTime'], errors='coerce')
DF['comments'] = DF['comments'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
DF['postTitle'] = DF['postTitle'].apply(lambda x: Preprocessing(str(x)))
DF['postDesc'] = DF['postDesc'].apply(lambda x: Preprocessing(str(x)))
DF['isNSFW'] = DF['isNSFW'].astype(bool)
print(DF.head())
print(DF.isnull().sum())

# Initialize graph and namespaces
g = rdflib.Graph()
SIOC = rdflib.Namespace("http://rdfs.org/sioc/ns#")
DCMI = rdflib.Namespace("http://purl.org/dc/elements/1.1/")
FOAF = rdflib.Namespace("http://xmlns.com/foaf/0.1/")
g.bind("sioc", SIOC)
g.bind("dc", DCMI)
g.bind("foaf", FOAF)

# Function to add posts to the graph
def add_post_to_graph(row, index, topic_uri):
    post_uri = rdflib.URIRef(f"http://reddit.com/post{index}")
    subreddit_uri = rdflib.URIRef(f"http://reddit.com/subreddit/{row['subReddit']}")
    author_uri = rdflib.URIRef(f"http://reddit.com/user/{row['authorName']}")
    
    g.add((post_uri, rdflib.RDF.type, SIOC.Post))
    g.add((post_uri, DCMI.title, rdflib.Literal(row['postTitle'])))
    g.add((post_uri, DCMI.description, rdflib.Literal(row['postDesc'])))
    g.add((post_uri, DCMI.date, rdflib.Literal(row['postTime'])))
    g.add((post_uri, SIOC.num_replies, rdflib.Literal(row['noOfUpvotes'])))
    g.add((post_uri, SIOC.link, rdflib.URIRef(row['postUrl'])))
    
    g.add((subreddit_uri, rdflib.RDF.type, SIOC.Container))
    g.add((subreddit_uri, SIOC.has_post, post_uri))
    g.add((post_uri, SIOC.Container, subreddit_uri))

    g.add((author_uri, rdflib.RDF.type, FOAF.Person))
    g.add((author_uri, FOAF.name, rdflib.Literal(row['authorName'])))
    g.add((post_uri, SIOC.has_creator, author_uri))

    comment_uri = rdflib.URIRef(f"http://reddit.com/comment/{index}")
    g.add((comment_uri, rdflib.RDF.type, SIOC.Comment))
    g.add((comment_uri, DCMI.title, rdflib.Literal(row['comments'])))
    g.add((post_uri, SIOC.has_reply, comment_uri))

    g.add((topic_uri, SIOC.has_post, post_uri))

# Group by subreddit and process each one for topic modeling
GroupedData = DF.groupby('subReddit')
all_topics = defaultdict(set)
for subreddit, group in GroupedData:
    print(f"Processing Subreddit: {subreddit}")
    group['Combined'] = group['postTitle'] + group['postDesc']
    
    group['Tokens'] = group['Combined'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x if word not in custom_stopwords])
    
    dictionary = corpora.Dictionary(group['Tokens'])
    corpus = [dictionary.doc2bow(text) for text in group['Tokens']]

    LDA = gensim.models.LdaModel(
        corpus,
        num_topics=5,
        id2word=dictionary,
        passes=15,
        iterations=100,
        random_state=42
    )
    
    for idx, topic in LDA.print_topics(num_words=5):
        topic_words = topic.split(" + ")
        unique_words = {word.split("*")[1].strip('"') for word in topic_words}
        all_topics[subreddit].update(unique_words)

# Save RDF graph in Turtle and JSON-LD formats
g.serialize('D:/FYP/Github/data-tails/Backend/Ontologies/KG1.ttl', format='turtle')
json_ld = g.serialize(format='json-ld', indent=4)
with open("D:/FYP/Github/data-tails/Backend/Ontologies/KG1.json", "w", encoding="utf-8") as f:
    f.write(json_ld)


subReddit           2
postTitle           0
postDesc        15607
postTime            0
authorName        587
noOfUpvotes         0
comments            0
noOfComments        2
postUrl             2
imageUrl            3
isNSFW              1
dtype: int64
   subReddit                                          postTitle postDesc  \
0  AskReddit                   [adult, especially, young, seem]       []   
1  AskReddit  [parent, consider, normal, consider, normal, a...       []   
2  AskReddit                                   [smell, comfort]       []   
3  AskReddit                       [history, best, time, woman]       []   
4  AskReddit                       [worst, way, someone, broke]       []   

             postTime            authorName noOfUpvotes  \
0 2024-08-06 01:02:35  Excellent-Studio7257        4068   
1 2024-08-06 01:47:22        Bigbumoffhappy        2073   
2 2024-08-05 22:21:53         bloomsmittenn        2188   
3 2024-08-06 03:32:59   More_food_please_77         

In [6]:
# Save subreddits and topics to Topics.csv
with open("D:/FYP/Github/data-tails/Backend/Ontologies/Topics.csv", mode='w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["subreddit", "topics"])
    for subreddit, topics in all_topics.items():
        writer.writerow([subreddit, ", ".join(sorted(topics))])

print("Subreddits and topics have been saved to Topics.csv.")



# List of subreddits to include in the new CSV
selected_subreddits = [
    "todayilearned", "news", "mildlyinteresting", "AskReddit", "Damnthatsinteresting", 
    "showerthoughts", "wallstreetbets", "NatureIsFuckingLit", "LifeProTips", "space", 
    "MovieDetails", "tifu", "futurology", "Music", "europe", "unpopularopinion", 
    "pakistan", "islamabad", "lahore", "karachi"
]

# Read the existing topics CSV
topics_df = pd.read_csv("D:/FYP/Github/data-tails/Backend/Ontologies/Topics.csv")

# Filter rows where 'subreddit' is in the selected subreddits list
filtered_topics_df = topics_df[topics_df['subreddit'].isin(selected_subreddits)]

# Save the filtered data to a new CSV file
filtered_topics_csv_path = "D:/FYP/Github/data-tails/Backend/Ontologies/FTopics.csv"
filtered_topics_df.to_csv(filtered_topics_csv_path, index=False)

print(f"Filtered topics saved to {filtered_topics_csv_path}")

Subreddits and topics have been saved to Topics.csv.
Filtered topics saved to D:/FYP/Github/data-tails/Backend/Ontologies/FTopics.csv


### Creating GRAPH STRUCTURE

In [ ]:
import rdflib
from rdflib import URIRef, Literal
from collections import defaultdict
import gensim
from gensim import corpora
import json

# RDF graph initialization
g = rdflib.Graph()
SIOC = rdflib.Namespace("http://rdfs.org/sioc/ns#")
DCMI = rdflib.Namespace("http://purl.org/dc/elements/1.1/")
FOAF = rdflib.Namespace("http://xmlns.com/foaf/0.1/")
REDDIT = rdflib.Namespace("http://reddit.com/ns#")  # Custom namespace for Reddit-specific relationships
g.bind("sioc", SIOC)
g.bind("dc", DCMI)
g.bind("foaf", FOAF)
g.bind("reddit", REDDIT)


# Function to add post data to RDF graph
def add_post_to_graph(row, index, topic_uri, subreddit_type_uri, author_type_uri, post_type_uri, comment_type_uri):
    post_uri = URIRef(f"http://reddit.com/post{index}")
    subreddit_uri = URIRef(f"http://reddit.com/subreddit/{row['subReddit']}")
    author_uri = URIRef(f"http://reddit.com/user/{row['authorName']}")
    
    # Add post properties and link to topic
    g.add((post_uri, rdflib.RDF.type, SIOC.Post))
    g.add((post_uri, DCMI.title, Literal(' '.join(row['postTitle']))))
    g.add((post_uri, DCMI.description, Literal(' '.join(row['postDesc']))))
    g.add((post_uri, DCMI.date, Literal(row['postTime'])))
    g.add((post_uri, SIOC.num_replies, Literal(row['noOfUpvotes'])))
    g.add((post_uri, SIOC.link, URIRef(row['postUrl'])))
    g.add((post_uri, SIOC.NSFW, Literal(row['isNSFW'])))
    g.add((post_uri, SIOC.has_type, post_type_uri))
    g.add((post_uri, SIOC.topic, topic_uri))

    # Create the relationships based on the list of relationship types
    # CreatedBy: Indicates that a post is created by an author.
    g.add((post_uri, REDDIT.CreatedBy, author_uri))

    # BelongsTo: Links a post to a subreddit.
    g.add((post_uri, REDDIT.BelongsTo, subreddit_uri))

    # PartOfPost: Indicates that a comment is part of a post (the comment belongs to the post).
    comment_uri = URIRef(f"http://reddit.com/comment/{index}")
    g.add((comment_uri, REDDIT.PartOfPost, post_uri))

    # HasType: Links an entity (post, comment) to its specific type (text post, link post, etc.).
    g.add((post_uri, REDDIT.HasType, post_type_uri))
    g.add((comment_uri, REDDIT.HasType, comment_type_uri))

    # TaggedIn: Indicates that a post or comment is tagged with a specific topic or keyword.
    for tag in row.get('tags', []):  # Assuming tags are available as a list
        tag_uri = URIRef(f"http://reddit.com/tag/{tag}")
        g.add((post_uri, REDDIT.TaggedIn, tag_uri))
        g.add((comment_uri, REDDIT.TaggedIn, tag_uri))

    # UpvotedBy: Indicates that a user has upvoted a post or comment.
    if row.get('upvotedBy'):
        for upvoter in row['upvotedBy']:
            upvoter_uri = URIRef(f"http://reddit.com/user/{upvoter}")
            g.add((post_uri, REDDIT.UpvotedBy, upvoter_uri))
            g.add((comment_uri, REDDIT.UpvotedBy, upvoter_uri))

    # DownvotedBy: Indicates that a user has downvoted a post or comment.
    if row.get('downvotedBy'):
        for downvoter in row['downvotedBy']:
            downvoter_uri = URIRef(f"http://reddit.com/user/{downvoter}")
            g.add((post_uri, REDDIT.DownvotedBy, downvoter_uri))
            g.add((comment_uri, REDDIT.DownvotedBy, downvoter_uri))

    # Moderates: Indicates that a user is a moderator of a subreddit.
    if row.get('moderators'):
        for moderator in row['moderators']:
            moderator_uri = URIRef(f"http://reddit.com/user/{moderator}")
            g.add((subreddit_uri, REDDIT.Moderates, moderator_uri))

    # HasFlair: Links a post or comment to a specific flair (e.g., tags like "Important", "Question", etc.).
    if row.get('flair'):
        flair_uri = URIRef(f"http://reddit.com/flair/{row['flair']}")
        g.add((post_uri, REDDIT.HasFlair, flair_uri))
        g.add((comment_uri, REDDIT.HasFlair, flair_uri))

    # Link post to subreddit and type
    g.add((subreddit_uri, rdflib.RDF.type, SIOC.Container))
    g.add((subreddit_uri, SIOC.has_post, post_uri))
    g.add((subreddit_uri, SIOC.has_type, subreddit_type_uri))
    g.add((post_uri, SIOC.Container, subreddit_uri))

    # Link to author and author type
    g.add((author_uri, rdflib.RDF.type, FOAF.Person))
    g.add((author_uri, FOAF.name, Literal(row['authorName'])))
    g.add((post_uri, SIOC.has_creator, author_uri))
    g.add((author_uri, SIOC.has_type, author_type_uri))

    # Add comments and link to comment type
    g.add((comment_uri, rdflib.RDF.type, SIOC.Comment))
    g.add((comment_uri, DCMI.title, Literal(row['comments'])))
    g.add((post_uri, SIOC.has_reply, comment_uri))
    g.add((comment_uri, SIOC.has_type, comment_type_uri))

# Process each subreddit and create LDA models
GroupedData = DF.groupby('subReddit')
all_topics = defaultdict(set)

for subreddit, group in GroupedData:
    group['Combined'] = group['postTitle'] + group['postDesc']
    group['Tokens'] = group['Combined'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x if word not in custom_stopwords])

    # Create dictionary and corpus
    dictionary = corpora.Dictionary(group['Tokens'])
    corpus = [dictionary.doc2bow(text) for text in group['Tokens']]
    LDA = gensim.models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15, iterations=100, random_state=42)
    
    for idx, topic in LDA.print_topics(num_words=5):
        topic_uri = URIRef(f"http://reddit.com/topic/{subreddit}_{idx}")
        topic_words = topic.split(" + ")
        unique_words = {word.split("*")[1].strip('"') for word in topic_words}
        all_topics[subreddit].update(unique_words)

        # Define URIs for types
        subreddit_type_uri = URIRef(f"http://reddit.com/type/subreddit/{subreddit}")
        post_type_uri = URIRef(f"http://reddit.com/type/post/{subreddit}")
        comment_type_uri = URIRef(f"http://reddit.com/type/comment/{subreddit}")
        author_type_uri = URIRef(f"http://reddit.com/type/author/{subreddit}")

        # Link subreddit to topic
        subreddit_uri = URIRef(f"http://reddit.com/subreddit/{subreddit}")
        g.add((subreddit_uri, SIOC.has_topic, topic_uri))
        g.add((topic_uri, rdflib.RDF.type, SIOC.Topic))

        # Iterate through each row in the group
        for index, row in group.iterrows():
            add_post_to_graph(row, index, topic_uri, subreddit_type_uri, author_type_uri, post_type_uri, comment_type_uri)

# Save graph
g.serialize('D:/FYP/Github/data-tails/Backend/Ontologies/KG.ttl', format='turtle')
json_ld = g.serialize(format='json-ld', indent=4)
with open("D:/FYP/Github/data-tails/Backend/Ontologies/KG.json", "w", encoding="utf-8") as f:
    f.write(json_ld)

## GraphRAG

In [ ]:
import json
import rdflib
import subprocess
import re
import time
from collections import defaultdict
from rdflib import Graph, Namespace, Literal, URIRef

# ✅ Define RDF Namespaces
SIOC = Namespace("http://rdfs.org/sioc/ns#")
DCMI = Namespace("http://purl.org/dc/elements/1.1/")
FOAF = Namespace("http://xmlns.com/foaf/0.1/")
REDDIT = Namespace("http://reddit.com/ns#")

# ✅ Load KG.json and Index Data for Fast Lookups
def load_kg_json(file_path):
    """Loads KG.json and creates a fast index for lookup by subreddit & topic."""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        print(f"✅ Loaded KG.json with {len(data)} entities.")

        # **Pre-index posts & comments for O(1) lookup**
        index = defaultdict(lambda: defaultdict(list))
        for entity_id, entity in data.items():
            subreddit = entity.get("sioc:Container", "").replace("http://reddit.com/subreddit/", "")
            topics = entity.get("sioc:topic", [])
            if isinstance(topics, str):
                topics = [topics]

            for topic in topics:
                topic = topic.replace("http://reddit.com/topic/", "")
                index[subreddit][topic].append(entity_id)

        return index, data

    except Exception as e:
        print(f"❌ Error loading KG.json: {str(e)}")
        return None, None

# ✅ Load KG.ttl & Build Fast Lookup Table
def load_kg_ttl(file_path):
    """Loads KG.ttl and builds an adjacency list for fast lookups."""
    try:
        g = Graph()
        g.parse(file_path, format="turtle")
        adjacency_list = defaultdict(list)

        for s, p, o in g:
            adjacency_list[str(s)].append((s, p, o))

        print(f"✅ Loaded KG.ttl with {len(g)} triples.")
        return g, adjacency_list
    except Exception as e:
        print(f"❌ Error loading KG.ttl: {str(e)}")
        return None, None

# ✅ **Optimized Comment Retrieval**
def retrieve_relevant_comments(index, kg_json, subreddit, topic):
    """Retrieves only relevant comments based on subreddit & topic index."""
    if not index:
        return "❌ KG.json not loaded."

    # **Step 1: Get relevant posts from pre-indexed data**
    relevant_posts = index.get(subreddit, {}).get(topic, [])
    if not relevant_posts:
        return "❌ No relevant posts found for the subreddit & topic."

    # **Step 2: Fetch only comments from those posts**
    matched_comments = []
    for post in relevant_posts:
        post_data = kg_json.get(post, {})
        if "sioc:has_reply" in post_data:
            replies = post_data["sioc:has_reply"]
            if isinstance(replies, str):
                replies = [replies]
            for comment in replies:
                comment_text = kg_json.get(comment, {}).get("dc:title", "")
                if comment_text:
                    matched_comments.append(comment_text)

    return "\n".join(matched_comments[:10]) if matched_comments else "❌ No relevant comments found."

# ✅ Query WizardLM2 (Fast)
def query_wizardlm2(context, user_query):
    """Queries WizardLM2 using Ollama API."""
    prompt = f"""
    Context:
    {context}

    Question:
    {user_query}

    Provide a detailed answer based on the context.
    """
    try:
        result = subprocess.run(
            ["ollama", "run", "wizardlm2"],
            input=prompt.encode("utf-8"),  
            capture_output=True
        )
        if result.returncode != 0:
            raise Exception(result.stderr.decode("utf-8"))
        return result.stdout.decode("utf-8").strip()
    except Exception as e:
        return f"Error querying WizardLM2: {str(e)}"

# ✅ **Run Main Program**
if __name__ == "__main__":
    kg_json_path = "./KG.json"
    kg_ttl_path = "./KG.ttl"

    print("\n🔍 Loading Knowledge Graphs...")
    index, kg_json = load_kg_json(kg_json_path)
    kg_ttl, adjacency_list = load_kg_ttl(kg_ttl_path)

    subreddit = "News"
    topic = "Democracy"
    user_query = "Middle District Court of Louisiana incident results?"

    print("\n🔍 Retrieving Relevant Comments...")
    start_time = time.time()
    context = retrieve_relevant_comments(index, kg_json, subreddit, topic)
    retrieval_time = round(time.time() - start_time, 2)
    print(f"✅ Retrieval Time: {retrieval_time} sec")
    print(f"Retrieved Context:\n{context}")

    print("\n🤖 Querying LLM...")
    response = query_wizardlm2(context, user_query)

    print("\n💡 LLM Response:", response)



🔍 Loading Knowledge Graphs...
✅ Loaded KG.json with 63763 entities.
❌ Error loading KG.json: 'list' object has no attribute 'items'


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x000001FD956EAE60>
Traceback (most recent call last):
  File "c:\Users\97156\AppData\Local\Programs\Python\Python310\lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "c:\Users\97156\AppData\Local\Programs\Python\Python310\lib\site-packages\isodate\isodatetime.py", line 55, in parse_datetime
    tmpdate = parse_date(datestring)
  File "c:\Users\97156\AppData\Local\Programs\Python\Python310\lib\site-packages\isodate\isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: 'Na'


✅ Loaded KG.ttl with 648909 triples.

🔍 Retrieving Relevant Comments...
✅ Retrieval Time: 0.0 sec

🤖 Querying LLM...

💡 LLM Response: The Middle District of Louisiana is one of the four federal judicial districts in the state of Louisiana. The court has jurisdiction over the following parishes:

* Avoyelles
* Bossier
* Catahoula
* Concordia
* East Carroll
* Franklin
* Grant
* La Salle
* Madison
* Natchitoches
* Ouachita
* Richland
* Ruston
* Tensas
* Union
* Vermilion
* Webster
* West Carroll

Since you're asking about the results of incidents in the Middle District of Louisiana, please specify the type of incident you're referring to (e.g. criminal, civil, defamation, etc.) and the approximate date of occurrence. I'll do my best to provide you with the relevant information.

Note: As a general rule, court records are public information, but some documents may be sealed or subject to redactions. I'll do my best to provide you with accurate and available information.


### Visualization Recommedation System

In [ ]:
import numpy as np
import spacy
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load NLP models
nlp = spacy.load("en_core_web_sm")
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define visualization categories
chart_types = {
    "area_chart": "Shows cumulative data trends with a filled area.",
    "bar_chart": "Used for comparing categories or ranking values.",
    "chord_diagram": "Best for visualizing relationships and interactions.",
    "circle_packing": "Represents hierarchical relationships in a compact form.",
    "connection_map": "Visualizes spatial relationships and geographic data.",
    "DAG": "Shows directed relationships, commonly used for processes or networks.",
    "donut_chart": "A variation of the pie chart, highlighting proportions.",
    "heatmap_chart": "Displays intensity values in a matrix format.",
    "line_chart": "Best for showing trends over time or sequential data.",
    "mosaic_plot": "Used to show the relationship between categorical variables.",
    "network_graph": "Illustrates complex relationships in networks.",
    "polar_area": "Represents cyclic data with proportionally scaled segments.",
    "small_multiples": "Facilitates comparisons across multiple categories.",
    "stacked_area_chart": "Shows part-to-whole relationships over time.",
    "sunburst_chart": "Depicts hierarchical data as concentric layers.",
    "tree_diagram": "Illustrates hierarchical relationships in tree structure.",
    "treemap_chart": "Depicts hierarchical structures using nested rectangles.",
    "voronoi_map": "Divides spatial regions based on distance.",
    "word_cloud": "Visualizes common words and keyword frequency in text-heavy data."
}

# Precompute chart category embeddings
category_embeddings = {
    chart: model.encode(description, normalize_embeddings=True)
    for chart, description in chart_types.items()
}

# Extract features from query & response
def extract_features(query, response):
    """Extracts key elements from the query-response pair."""
    combined_text = query + " " + response
    doc = nlp(combined_text)

    numbers = [token.text for token in doc if token.like_num]
    locations = [ent.text for ent in doc.ents if ent.label_ in {"GPE", "LOC"}]
    
    trend_keywords = {"increase", "decline", "growth", "rise", "drop", "trend", "fall", "reduce", "expansion"}
    trends = [token.lemma_ for token in doc if token.lemma_ in trend_keywords]
    
    relationship_keywords = {"prefer", "dominate", "compared", "versus", "majority", "minority"}
    relationships = [token.lemma_ for token in doc if token.lemma_ in relationship_keywords]
    
    ranking_keywords = {"ranking", "top", "best", "percent", "favorite", "most popular"}
    rankings = [token.lemma_ for token in doc if token.lemma_ in ranking_keywords]
    
    geo_terms = {"earthquake", "magnitude", "epicenter", "seismic", "hurricane", "flood", "storm"}
    geo_features = [token.lemma_ for token in doc if token.lemma_ in geo_terms]
    
    keywords = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Identify text-heavy responses
    is_text_heavy = len(keywords) > 30 

    return {
        "numbers": numbers,
        "locations": locations,
        "trends": trends,
        "relationships": relationships,
        "geo_features": geo_features,
        "rankings": rankings,
        "keywords": keywords,
        "is_text_heavy": is_text_heavy  # New feature
    }

# Adjust scores based on extracted features
def boost_scores(scores, response_type, features):
    """Boosts scores dynamically based on detected features."""
    boosts = {
        "numeric": {"bar_chart": 0.6, "line_chart": 0.5, "heatmap_chart": 0.4},
        "geographic": {"connection_map": 0.7, "voronoi_map": 0.6, "network_graph": 0.5},
        "textual": {"word_cloud": 0.7, "chord_diagram": 0.5}
    }
    
    if response_type in boosts:
        for chart, boost in boosts[response_type].items():
            scores[chart] += boost
    
    if features["trends"]:
        scores["line_chart"] += 0.6
        scores["area_chart"] += 0.5
    
    if features["relationships"]:
        scores["bar_chart"] += 0.6
        scores["chord_diagram"] += 0.5
    
    if features["geo_features"]:
        scores["connection_map"] += 0.7
        scores["network_graph"] += 0.6
    
    if features["rankings"]:
        scores["bar_chart"] += 0.7
        scores["mosaic_plot"] += 0.5

    # Ensure Word Cloud is boosted for text-heavy responses
    if features["is_text_heavy"]:
        scores["word_cloud"] += 1.0

    return scores

# Function to recommend all visualizations
def recommend_visualizations(query, response):
    """Recommends a ranked list of all visualizations."""
    features = extract_features(query, response)
    response_embedding = model.encode(query + " " + response, normalize_embeddings=True)
    
    similarity_scores = {
        chart: cosine_similarity([response_embedding], [embedding]).flatten()[0]
        for chart, embedding in category_embeddings.items()
    }
    
    # Apply feature-based score boosts
    similarity_scores = boost_scores(similarity_scores, "numeric", features)

    # Normalize scores for better ranking
    min_score = min(similarity_scores.values())
    max_score = max(similarity_scores.values())
    if max_score - min_score > 0:  # Avoid division by zero
        for chart in similarity_scores:
            similarity_scores[chart] = (similarity_scores[chart] - min_score) / (max_score - min_score)

    # Sort and return all charts
    ranked_charts = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)[:4]
    
    return [(chart, round(score, 2)) for chart, score in ranked_charts]

def getViz(user_query, response):
    recommended_charts = recommend_visualizations(user_query, response)
    print("\n✅ Ranked Recommended Visualizations:")
    for chart, score in recommended_charts:
        print(f"{chart}: {score}")

getViz(user_query, response)


c:\Users\97156\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)



✅ Top 3 Recommended Visualizations: [('bar_chart', 0.68), ('line_chart', 0.63), ('heatmap_chart', 0.46)]


c:\Users\97156\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
